_Updated date: November 11, 2025_

# 🎓 Databricks Workshop: HQRI Risk Adjustment & Analytics
**Healthcare Quality Reporting & Improvement - DBSQL Analytics Workshop**

---

## 📚 Workshop Objectives

By the end of this workshop, you will be able to:

1. ✅ Build **Medallion Architecture** pipelines for risk adjustment data
2. ✅ Calculate **HCC (Hierarchical Condition Category) risk scores** for Medicare Advantage
3. ✅ Create **encounter datamart** tables for CMS submissions
4. ✅ Perform **data quality audits** for regulatory compliance
5. ✅ Build **Gold layer analytics** for revenue impact and Star Ratings
6. ✅ Forecast risk scores using **Databricks SQL (DBSQL)**

---

## 🏥 HQRI Use Case: Medicare Risk Adjustment

**HQRI (Healthcare Quality Reporting & Improvement)** manages Humana's risk adjustment and quality reporting for Medicare Advantage:

- 💰 **Risk Score Calculations**: Determine CMS payments based on member health status
- 📊 **Encounter Datamart**: Aggregate and validate claims/encounters for CMS
- ⭐ **Star Ratings**: Drive quality metrics that impact bonus payments
- ✅ **Compliance & Audits**: Ensure federal regulatory compliance
- 📈 **Revenue Optimization**: Accurate coding and forecasting maximize financial performance

### 🗂️ Dataset Overview

We'll work with **Medicare risk adjustment data**:
- **Members**: Medicare Advantage enrollees with demographics
- **Claims**: Medical encounters with diagnosis codes
- **Diagnoses**: ICD-10 codes mapped to HCC categories
- **Providers**: Healthcare providers submitting encounters
- **Procedures**: Services rendered and billed

---




# What is a lakehouse?

1. **Hybrid Architecture:**  
   A lakehouse combines the best of data lakes (flexible, cheap storage) and data warehouses (structured, fast analytics), providing transactional and governance features on top of open cloud storage.

2. **ACID Transactions and Schema Governance:**  
   Lakehouses support ACID transactions for consistent concurrent data access and enforce schema management, which is essential for data integrity and compliance.

3. **Open and Decoupled:**  
   They use open file formats (like Parquet), decouple compute from storage for flexible scalability, and allow access by a variety of analytics, BI, and machine learning tools.

4. **Supports All Workloads and Data Types:**  
   The architecture enables SQL analytics, data science, machine learning, and can handle structured, semi-structured, and unstructured data (including images, text, video).

5. **Single Platform, Enterprise Ready:**  
   With features like real-time streaming, end-to-end governance, access control, and data discovery tools, lakehouses reduce complexity—allowing enterprises to manage all data and analytics needs in one unified system.
![](https://www.databricks.com/wp-content/uploads/2020/01/data-lakehouse-new.png)

## Unity Catalog

[Unified and open governance for data and AI in the Lakehouse](https://www.databricks.com/product/unity-catalog#features)

Eliminate silos, simplify governance and accelerate insights at scale:

- Centralizes governance, access control, auditing, and data discovery for all data and AI assets across Databricks workspaces.
- Enables fine-grained, consistent data access policies (row- and column-level), defined once and applied everywhere.
- Provides comprehensive data lineage and audit logs, showing how and by whom data is accessed and transformed.
- Supports data discovery, tagging, and documentation, making it easier to find and understand datasets and models.
- Works across multiple clouds and supports open formats (Delta, Parquet, etc.), avoiding vendor lock-in and enabling broad interoperability.
- Allows secure data and AI sharing within and outside the organization, including clean rooms and partner collaborations.
- Provides built-in monitoring for data quality, freshness, and usage, helping ensure compliance and rapid troubleshooting.
- Integrates tightly with the catalog/schema/object model, enhancing organization and security for all managed data assets.

![](https://www.databricks.com/sites/default/files/2025-05/header-unity-catalog.png?v=1748513086)

[Unity Catalog Search & Data Explorer](https://app.getreprise.com/launch/96mpAqy/)

[Exploring Lineage and Governance with Unity Catalog](https://app.getreprise.com/launch/MnqjQDX/)

[A Comprehensive Guide to Data and AI Governance](https://www.databricks.com/sites/default/files/2024-08/comprehensive-guide-to-data-and-ai-governance.pdf)






## Medallion lakehouse architecture

In this example, we will be following the **medallion lakehouse architecture**. The medallion architecture is a data design pattern to organize data in a lakehouse. The goal is to progressively improve the quality and structure of the data as it flows through each layer (Bronze [**raw**] → Silver [**staging**] → Gold [**main**]).

1. **Bronze layer**: the raw, unvalidated data
2. **Silver**: cleansed and conformed data
3. **Gold**: curated business-level tables

<img src="https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922" alt="Managed Tables" width="600" height="500">

## Managed tables

[How Unity Catalog Managed Tables Automate Performance at Scale](https://www.databricks.com/blog/how-unity-catalog-managed-tables-automate-performance-scale) with [Predictive Optimization](https://learn.microsoft.com/en-us/azure/databricks/optimizations/predictive-optimization)


<!-- ![](https://www.databricks.com/sites/default/files/inline-images/image2_48.png?v=1751297384) -->

<img src="https://www.databricks.com/sites/default/files/inline-images/image2_48.png?v=1751297384" alt="Managed Tables" width="600" height="500">


[Faster Queries: 20X query latency reduction](https://www.databricks.com/blog/predictive-optimization-automatically-delivers-faster-queries-and-lower-tco)

**Predictive Optimization** in Databricks automates table management by leveraging Unity Catalog and the Data Intelligence Platform. This innovative feature currently runs the following optimizations for Unity Catalog managed tables:

* **OPTIMIZE** - Triggers incremental clustering for enabled tables. Improves query performance by optimizing file sizes.
* **VACUUM** - Reduces storage costs by deleting data files no longer referenced by the table.
* **ANALYZE** - Triggers incremental update of statistics to improve query performance. 


<img src="https://www.databricks.com/sites/default/files/styles/max_1000x1000/public/2024-05/db-976-blog-img-og.png?itok=qWBT8VA-&v=1717158571" alt="Managed Tables" width="600" height="500">

**Compaction** - This enhances query performance by optimizing file sizes, ensuring that data retrieval is efficient.

**Liquid Clustering** - This technique incrementally clusters incoming data, enabling optimal data layout and efficient data skipping.



# Databricks Medallion Pipeline for a Healthcare Payer


## Risk Adjustment & HCC Modeling Concepts

### 🏥 Medicare Risk Adjustment Overview

CMS uses **HCC (Hierarchical Condition Category)** models to adjust capitation payments based on beneficiary health status:

1. **ICD-10 Diagnosis Codes** → mapped to **HCC categories**
2. **HCC categories** → assigned **coefficient weights**
3. **Risk Score** = Sum of HCC weights + demographic factors (age, sex, disability status)
4. **Payment** = Base rate × Risk Score

**Example:**
- Member with diabetes (HCC 19, weight 0.318) and heart failure (HCC 85, weight 0.368)
- Base demographic score: 0.500
- **Total Risk Score = 0.500 + 0.318 + 0.368 = 1.186**
- If base payment = $10,000, **CMS pays: $10,000 × 1.186 = $11,860**

### ⭐ CMS Star Ratings Impact

Star Ratings (1-5 stars) impact bonus payments:
- **5 stars**: Up to 5% bonus payment
- **4+ stars**: Quality Bonus Payments (QBP)
- Ratings based on quality measures (clinical outcomes, patient experience, access)

### 📊 HQRI Data Model

For risk adjustment, key tables include:
- **Encounters/Claims**: Medical services with diagnosis codes
- **Diagnosis-to-HCC Mapping**: ICD-10 → HCC crosswalk
- **HCC Coefficients**: CMS model weights by year
- **Members**: Demographics and enrollment status
- **Risk Scores**: Calculated member-level risk scores

<div style="display: flex; justify-content: space-between;">
  <img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/5c87faea-3e60-4f71-826d-42d04f6cdc0b.png" alt="Dimensional Model" width="400" height="350">
  <img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/6826c275-d462-4c07-a978-43fe9c40f3ed.png" alt="Data Vault" width="400" height="350">
</div>

**Resources:**
- [CMS Risk Adjustment Model](https://www.cms.gov/medicare/payment/medicare-advantage-rates-statistics/risk-adjustment)
- [Implementing Dimensional Modeling on Databricks](https://www.databricks.com/blog/implementing-dimensional-data-warehouse-databricks-sql-part-1)






# SETUP

Just run next couple of cells for setup!

In [0]:
dbutils.widgets.text("catalog", "sunmin_catalog", "Catalog")
dbutils.widgets.text("bronze_db", "payer_bronze", "Bronze DB")
dbutils.widgets.text("silver_db", "payer_silver", "Silver DB")
dbutils.widgets.text("gold_db", "payer_gold", "Gold DB")

catalog = dbutils.widgets.get("catalog")
bronze_db = dbutils.widgets.get("bronze_db")
silver_db = dbutils.widgets.get("silver_db")
gold_db = dbutils.widgets.get("gold_db")

path = f"/Volumes/{catalog}/{bronze_db}/payer/files/"

print(f"Catalog: {catalog}")
print(f"Bronze DB: {bronze_db}")
print(f"Silver DB: {silver_db}")
print(f"Gold DB: {gold_db}")
print(f"Path: {path}")

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bronze_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {silver_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")

spark.sql(f"CREATE VOLUME IF NOT EXISTS {bronze_db}.payer")

# Create the volume and folders
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/claims")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/diagnosis")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/procedures")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/members")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/files/providers")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payer/downloads")

In [0]:
import requests
import zipfile
import io
import os
import shutil

# Define the URL of the ZIP file
url = "https://github.com/bigdatavik/databricksfirststeps/blob/6b225621c3c010a2734ab604efd79c15ec6c71b8/data/Payor_Archive.zip?raw=true"

# Download the ZIP file
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Define the base path
base_path = f"/Volumes/{catalog}/{bronze_db}/payer/downloads" 

# Extract the ZIP file to the base path
zip_file.extractall(base_path)

# Define the paths
paths = {
    "claims.csv": f"{base_path}/claims",
    "diagnoses.csv": f"{base_path}/diagnosis",
    "procedures.csv": f"{base_path}/procedures",
    "member.csv": f"{base_path}/members",
    "providers.csv": f"{base_path}/providers"
}

# Create the destination directories if they do not exist
for dest_path in paths.values():
    os.makedirs(dest_path, exist_ok=True)

# Move the files to the respective directories
for file_name, dest_path in paths.items():
    source_file = f"{base_path}/{file_name}"
    if os.path.exists(source_file):
        os.rename(source_file, f"{dest_path}/{file_name}")


# Copy the files to the specified directories and print the paths
shutil.copy(f"{base_path}/claims/claims.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/claims/claims.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/claims/claims.csv")

shutil.copy(f"{base_path}/diagnosis/diagnoses.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/diagnosis/diagnosis.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/diagnosis/diagnosis.csv")

shutil.copy(f"{base_path}/procedures/procedures.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/procedures/procedures.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/procedures/procedures.csv")

shutil.copy(f"{base_path}/members/member.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/members/members.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/members/members.csv")

shutil.copy(f"{base_path}/providers/providers.csv", f"/Volumes/{catalog}/{bronze_db}/payer/files/providers/providers.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payer/files/providers/providers.csv")



# 🚀 Let's Build Your First Data Pipeline!

---

## Workshop Roadmap

```
📥 Bronze Layer    →    🔧 Silver Layer    →    ⭐ Gold Layer    →    📊 Analytics
   (Raw Data)          (Cleaned Data)        (Business Tables)      (Insights)
```

In the following sections, we'll build a complete data pipeline following the **Medallion Architecture**:

1. **Bronze Layer**: Ingest raw CSV files into Delta tables
2. **Silver Layer**: Clean, deduplicate, and transform data
3. **Gold Layer**: Create enriched analytics tables
4. **Analytics**: Generate insights and visualizations

Let's get started! 🎉

# 📥 Bronze/Silver Layers – Streamlined Data Preparation

---

## Overview: Simplified Bronze & Silver

For HQRI analytics, we'll **streamline** Bronze and Silver layers to quickly get to Gold layer insights:

### Bronze Layer (Raw Data Landing)
- 📂 Ingest encounter data "as-is" using `COPY INTO`
- 💾 Store in Delta Lake for audit trails
- ⏱️ Maintain full history for compliance

### Silver Layer (Clean & Validate)
- 🧹 Remove duplicates and validate data quality
- 🔄 Map ICD-10 codes to HCC categories
- ✅ Apply business rules for CMS submission eligibility

> **💡 Focus**: We'll execute Bronze/Silver steps efficiently so we can spend more time on **Gold layer analytics** that drive business value!

---



## Step 1: Verify Source Files

Let's first check that our source files are available:

In [0]:
%sql
LIST '/Volumes/sunmin_catalog/payer_bronze/payer/files/claims/'

## Step 2: Load Data with COPY INTO

### 📖 Understanding COPY INTO

`COPY INTO` is Databricks' recommended command for loading data from cloud storage into Delta tables.

**Key Benefits:**
- ✅ **Idempotent**: Safely re-run without duplicating data
- ✅ **Incremental**: Only loads new files automatically
- ✅ **Schema Evolution**: Can merge new columns with `mergeSchema` option
- ✅ **Atomic**: Either succeeds completely or rolls back

**Syntax:**
```sql
COPY INTO <table_name>
FROM '<source_path>'
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS('mergeSchema' = 'true')
```

📚 **Learn More:**
- [COPY INTO Documentation](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into)
- [COPY INTO Examples](https://learn.microsoft.com/en-us/azure/databricks/ingestion/cloud-object-storage/copy-into/)


### Loading Data with SQL

In [0]:
%sql
-- Load Claims Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.claims_raw;
COPY INTO payer_bronze.claims_raw FROM
(SELECT
*
FROM '/Volumes/sunmin_catalog/payer_bronze/payer/files/claims/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true', 'force' = 'true');

-- NOTE: 'force = true' is used here for demo purposes only to reload all files every time. In production, omit this option so COPY INTO only processes new data files.


-- Load Diagnosis Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.diagnosis_raw;
COPY INTO payer_bronze.diagnosis_raw FROM
(SELECT
*
FROM '/Volumes/sunmin_catalog/payer_bronze/payer/files/diagnosis/')

FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Members Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.members_raw;
COPY INTO payer_bronze.members_raw FROM
(SELECT
*
FROM '/Volumes/sunmin_catalog/payer_bronze/payer/files/members/')

FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Procedures Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.procedures_raw;
COPY INTO payer_bronze.procedures_raw FROM
(SELECT
*
FROM '/Volumes/sunmin_catalog/payer_bronze/payer/files/procedures/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


-- Load Providers Data into Bronze Table
CREATE TABLE IF NOT EXISTS payer_bronze.providers_raw;
COPY INTO payer_bronze.providers_raw FROM
(SELECT
*
FROM '/Volumes/sunmin_catalog/payer_bronze/payer/files/providers/')
FILEFORMAT = CSV
FORMAT_OPTIONS('header' = 'true',
               'inferSchema' = 'true',
               'delimiter' = ',')
COPY_OPTIONS ('mergeSchema' = 'true');


### 🐍 Alternative: Loading Data with PySpark

While SQL is great for batch loading, PySpark gives you more programmatic control. Here's how to load the same data using PySpark:

In [0]:
# Example: Load data using PySpark
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

# Option 1: Let Spark infer the schema
claims_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/sunmin_catalog/payer_bronze/payer/files/claims/")

# Display first 10 rows
display(claims_df.limit(10))

# Show schema
print("Claims Schema:")
claims_df.printSchema()

# Get row count
print(f"\nTotal rows loaded: {claims_df.count()}")

# Write to Delta table (this creates or replaces the table)
# claims_df.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .saveAsTable("payer_bronze.claims_raw_pyspark")


## Silver Layer – Transforms for Risk Adjustment

Now let's clean the data and prepare it for HCC risk scoring:

**Key Silver Transformations:**
- ✅ Deduplicate encounters
- ✅ Validate diagnosis codes (ICD-10 format)
- ✅ Filter to eligible members (Medicare Advantage, active enrollment)
- ✅ Map diagnoses to HCC categories

---

> **💡 Speed Focus**: We'll run these transforms to move to Gold layer analytics!



## Step 1: Transform Bronze to Silver (SQL)

Let's clean and transform our Bronze tables. We'll demonstrate with multiple examples using both **SQL** and **PySpark**.

In [0]:
%sql
-- Create silver schema
CREATE SCHEMA IF NOT EXISTS payer_silver;


-- Members: select relevant fields, cast types, remove duplicates
CREATE OR REPLACE TABLE payer_silver.members AS
SELECT
  DISTINCT CAST(member_id AS STRING) AS member_id,
  TRIM(first_name) AS first_name,
  TRIM(last_name) AS last_name,
  CAST(birth_date AS DATE) AS birth_date,
  gender,
  plan_id,
  CAST(effective_date AS DATE) AS effective_date
FROM payer_bronze.members_raw
WHERE member_id IS NOT NULL;


-- Claims: remove duplicates, prepare data
CREATE OR REPLACE TABLE payer_silver.claims AS
SELECT
  DISTINCT claim_id,
  member_id,
  provider_id,
  CAST(claim_date AS DATE) AS claim_date,
  ROUND(total_charge, 2) AS total_charge,
  LOWER(claim_status) AS claim_status
FROM payer_bronze.claims_raw
WHERE claim_id IS NOT NULL AND total_charge > 0;


-- Providers: deduplicate
CREATE OR REPLACE TABLE payer_silver.providers AS
SELECT
  DISTINCT provider_id,
  npi,
  provider_name,
  specialty,
  address,
  city,
  state
FROM payer_bronze.providers_raw
WHERE provider_id IS NOT NULL;


## Step 2: Transform with PySpark

Now let's see how to do the same transformations using PySpark. This approach is more flexible for complex business logic.

### Example: Transform Procedures Table with PySpark


In [0]:
from pyspark.sql.functions import col, trim, upper, round as spark_round, when, regexp_replace

# Read from Bronze
procedures_bronze = spark.table("payer_bronze.procedures_raw")

# Clean and cast the amount column
procedures_bronze_clean = procedures_bronze.withColumn(
    "amount_clean",
    regexp_replace(col("amount"), "[^0-9.]", "").cast("double")
)

# Apply transformations
procedures_silver = procedures_bronze_clean \
    .dropDuplicates(['claim_id', 'procedure_code']) \
    .filter(col("claim_id").isNotNull()) \
    .filter(col("amount_clean") > 0) \
    .select(
        col("claim_id"),
        upper(trim(col("procedure_code"))).alias("procedure_code"),
        trim(col("procedure_desc")).alias("procedure_desc"),
        spark_round(col("amount_clean"), 2).alias("amount"),
        when(col("amount_clean") < 100, "Low")
        .when(col("amount_clean") < 500, "Medium")
        .when(col("amount_clean") < 1000, "High")
        .otherwise("Very High").alias("cost_category")
    )

# Show sample data
print("Transformed Procedures (first 10 rows):")
display(procedures_silver.limit(10))

# Show statistics
print("\nCost Category Distribution:")
display(procedures_silver.groupBy("cost_category").count().orderBy("cost_category"))

# Write to Silver table
procedures_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("payer_silver.procedures")


# 🤖 Using Databricks AI Assistant

---

Databricks AI Assistant can help you write code, understand data, and troubleshoot issues!

### How to Use AI Assistant:
1. Click the AI Assistant icon
2. Ask questions in natural language
3. Get code suggestions and explanations

### Example Prompts to Try:
- "How do I calculate the total claims by specialty?"
- "Show me how to create a window function for running totals"
- "What does spark.table() command do?"
- "Help me debug this PySpark error"

---



## 🎯 YOUR TURN! (3 mins)
Ask Databricks Assistant: "How do I calculate the total claims by specialty in SQL?"

In [0]:
%sql
SELECT
    claim_status AS specialty,
    SUM(total_charge) AS total_claims
FROM sunmin_catalog.payer_silver.claims
GROUP BY claim_status

---

## 🔄 Why Migrate from SAS to Databricks?

### Common HQRI Challenges with SAS

Healthcare organizations using SAS for risk adjustment analytics face several limitations:

1. **🐌 Performance Bottlenecks**
   - Single-server processing limits scalability
   - Large member populations (millions of records) cause memory issues
   - Batch processing takes hours or overnight

2. **💰 Cost Concerns**
   - Expensive annual licensing fees
   - Additional costs for SAS/ACCESS, SAS Enterprise Guide, SAS Visual Analytics
   - Hardware upgrades needed for growing data volumes

3. **🔧 Development Complexity**
   - Multiple PROC steps required for simple operations
   - Limited modern SQL features (no EXPLODE, limited window functions)
   - SAS macros difficult to maintain and debug
   - Separate tools needed for visualization and dashboards

4. **☁️ Cloud Migration Challenges**
   - Legacy on-premises architecture
   - Difficult integration with cloud data lakes
   - Limited real-time analytics capabilities

### Databricks Advantages for HQRI

| **Capability** | **Impact** |
|----------------|------------|
| **Distributed Processing** | Handle billions of encounters with sub-second queries |
| **Modern SQL** | Window functions, CTEs, EXPLODE - cleaner, more maintainable code |
| **Unified Platform** | SQL, Python, R, dashboards, ML - all in one place |
| **Delta Lake** | ACID transactions + time travel for audit compliance |
| **Unity Catalog** | Row/column-level security for PHI/PII protection |
| **Real-time Analytics** | Streaming + batch unified for immediate insights |
| **Cost Efficiency** | Pay-per-use vs. fixed licensing, auto-scaling |


---


# ⭐ Gold Layer – HQRI Risk Adjustment Analytics

---

## What is the HQRI Gold Layer?

The **Gold Layer** is where we deliver **business value** for HQRI. Here we create analytics tables that directly support:

- 💰 **Risk Score Calculations**: Member-level HCC risk scores for CMS payments
- 📊 **Encounter Datamart**: CMS submission-ready data with quality validations
- ⭐ **Star Ratings**: Quality metrics that impact bonus payments
- 📈 **Revenue Forecasting**: Projected payments based on risk scores
- ✅ **Compliance Audits**: Data quality metrics for regulatory requirements
- 👥 **Member Stratification**: High-risk vs. low-risk population segmentation
- 🏥 **Provider Performance**: Risk capture rates by provider

> **🎯 HQRI Value**: Each Gold table directly answers a business question that impacts revenue, compliance, or quality!

---

# Sunmin TODO: Add hands-on examples from Gold

## Example 1: HCC Risk Score Calculation

### 🎯 Business Goal
Calculate member-level risk scores based on HCC categories to determine CMS payments.

**Risk Score Formula:**
```
Risk Score = Demographic Score + Σ(HCC Coefficients)
```

### 📊 SAS vs. Databricks Comparison

Let's compare how this is traditionally done in **SAS** vs. **Databricks SQL** and **PySpark**:

---

#### Traditional SAS Approach

```sas
/* SAS: HCC Risk Score Calculation */
/* Step 1: Create HCC reference table */
DATA work.hcc_reference;
    INPUT icd10_code $ 1-10 diagnosis_desc $ 12-50 hcc_category hcc_coefficient;
    DATALINES;
E11.9      Type 2 Diabetes                    19  0.318
I50.9      Heart Failure                      85  0.368
I10        Hypertension                       0   0.000
J44.9      COPD                              111  0.328
N18.3      CKD Stage 3                       138  0.237
;
RUN;

/* Step 2: Merge claims with diagnoses and HCC mapping */
PROC SQL;
    CREATE TABLE work.diagnosis_with_hcc AS
    SELECT 
        d.claim_id,
        d.diagnosis_code,
        d.diagnosis_desc,
        h.hcc_category,
        h.hcc_coefficient,
        CASE 
            WHEN h.hcc_category IS NOT NULL AND h.hcc_category > 0 
            THEN 1 ELSE 0 
        END AS is_hcc
    FROM work.diagnosis_raw AS d
    LEFT JOIN work.hcc_reference AS h
        ON UPCASE(STRIP(d.diagnosis_code)) = UPCASE(STRIP(h.icd10_code));
QUIT;

/* Step 3: Calculate member-level risk scores */
PROC SQL;
    CREATE TABLE work.member_hccs AS
    SELECT DISTINCT
        c.member_id,
        m.first_name,
        m.last_name,
        m.birth_date,
        m.gender,
        m.plan_id,
        dh.hcc_category,
        dh.hcc_coefficient,
        dh.diagnosis_code,
        YEAR(TODAY()) - YEAR(m.birth_date) AS age
    FROM work.claims AS c
    INNER JOIN work.members AS m 
        ON c.member_id = m.member_id
    INNER JOIN work.diagnosis_with_hcc AS dh 
        ON c.claim_id = dh.claim_id
    WHERE dh.is_hcc = 1;
QUIT;

/* Step 4: Aggregate and calculate final risk scores */
PROC SQL;
    CREATE TABLE work.member_risk_scores AS
    SELECT 
        member_id,
        first_name,
        last_name,
        birth_date,
        gender,
        plan_id,
        age,
        CASE 
            WHEN age < 65 THEN 0.350
            WHEN age BETWEEN 65 AND 69 THEN 0.450
            WHEN age BETWEEN 70 AND 74 THEN 0.550
            WHEN age BETWEEN 75 AND 79 THEN 0.650
            ELSE 0.750
        END AS demographic_score,
        SUM(hcc_coefficient) AS hcc_score,
        COUNT(DISTINCT hcc_category) AS hcc_count,
        CALCULATED demographic_score + CALCULATED hcc_score AS total_risk_score,
        ROUND((CALCULATED demographic_score + CALCULATED hcc_score) * 10000, 0.01) 
            AS projected_annual_payment
    FROM work.member_hccs
    GROUP BY member_id, first_name, last_name, birth_date, gender, plan_id, age
    ORDER BY total_risk_score DESC;
QUIT;
```

**SAS Challenges:**
- ❌ Multiple PROC SQL steps required
- ❌ Intermediate tables clutter WORK library
- ❌ Limited scalability with large datasets
- ❌ No automatic optimization or parallelization
- ❌ Complex syntax for array aggregations (HCC categories list)

---

Now let's see how **Databricks SQL** simplifies this!

In [0]:
%sql
-- Create gold schema
CREATE SCHEMA IF NOT EXISTS payer_gold;

-- Step 1a: Create HCC Mapping Reference Table (simulated for demo)
CREATE OR REPLACE TABLE payer_gold.hcc_reference (
  icd10_code STRING,
  diagnosis_desc STRING,
  hcc_category INT,
  hcc_coefficient DOUBLE
);

INSERT INTO payer_gold.hcc_reference VALUES
('E11.9', 'Type 2 Diabetes', 19, 0.318),
('I50.9', 'Heart Failure', 85, 0.368),
('I10', 'Hypertension', 0, 0.000),
('J44.9', 'COPD', 111, 0.328),
('N18.3', 'CKD Stage 3', 138, 0.237),
('F32.9', 'Depression', 59, 0.309),
('E78.5', 'Hyperlipidemia', 0, 0.000),
('I25.10', 'CAD', 88, 0.184);

-- Step 1b: Join Diagnoses to HCC Categories
CREATE OR REPLACE TABLE payer_gold.diagnosis_with_hcc AS
SELECT
  d.claim_id,
  d.diagnosis_code,
  d.diagnosis_desc,
  h.hcc_category,
  h.hcc_coefficient,
  CASE WHEN h.hcc_category IS NOT NULL AND h.hcc_category > 0 THEN 1 ELSE 0 END as is_hcc
FROM payer_bronze.diagnosis_raw d
LEFT JOIN payer_gold.hcc_reference h 
  ON UPPER(TRIM(d.diagnosis_code)) = UPPER(TRIM(h.icd10_code));

-- Step 1c: Calculate Member-Level HCC Risk Scores
CREATE OR REPLACE TABLE payer_gold.member_risk_scores AS
WITH member_hccs AS (
  SELECT DISTINCT
    c.member_id,
    m.first_name,
    m.last_name,
    m.birth_date,
    m.gender,
    m.plan_id,
    dh.hcc_category,
    dh.hcc_coefficient,
    dh.diagnosis_code,
    YEAR(CURRENT_DATE()) - YEAR(m.birth_date) as age
  FROM payer_silver.claims c
  INNER JOIN payer_silver.members m ON c.member_id = m.member_id
  INNER JOIN payer_gold.diagnosis_with_hcc dh ON c.claim_id = dh.claim_id
  WHERE dh.is_hcc = 1
),
member_scores AS (
  SELECT
    member_id,
    first_name,
    last_name,
    birth_date,
    gender,
    plan_id,
    age,
    CASE 
      WHEN age < 65 THEN 0.350
      WHEN age BETWEEN 65 AND 69 THEN 0.450
      WHEN age BETWEEN 70 AND 74 THEN 0.550
      WHEN age BETWEEN 75 AND 79 THEN 0.650
      ELSE 0.750
    END as demographic_score,
    SUM(hcc_coefficient) as hcc_score,
    COUNT(DISTINCT hcc_category) as hcc_count,
    COLLECT_SET(hcc_category) as hcc_categories,
    COLLECT_SET(diagnosis_code) as diagnosis_codes
  FROM member_hccs
  GROUP BY member_id, first_name, last_name, birth_date, gender, plan_id, age
)
SELECT
  *,
  demographic_score + hcc_score as total_risk_score,
  ROUND((demographic_score + hcc_score) * 10000, 2) as projected_annual_payment
FROM member_scores
ORDER BY total_risk_score DESC;

### 🚀 Databricks SQL Advantages

**Databricks SQL Benefits:**
- ✅ **Single SQL Statement**: All logic in one CREATE TABLE AS with CTEs
- ✅ **Advanced Functions**: COLLECT_SET() for array aggregation (no SAS equivalent)
- ✅ **Automatic Optimization**: Query engine optimizes joins and aggregations
- ✅ **Scalability**: Distributed processing handles billions of rows
- ✅ **Unity Catalog**: Built-in governance, lineage tracking, and access control
- ✅ **Delta Lake**: ACID transactions, time travel, schema evolution
- ✅ **Real-time Refresh**: Can be scheduled or triggered automatically

### 💡 PySpark Alternative

For complex business logic or programmatic control, use PySpark:


In [0]:
# PySpark Example: HCC Risk Score Calculation
from pyspark.sql.functions import (
    col, year, current_date, sum as _sum, count, countDistinct,
    collect_set, when, round as spark_round, lit
)

# Read tables
claims = spark.table("payer_silver.claims")
members = spark.table("payer_silver.members")
diagnosis_with_hcc = spark.table("payer_gold.diagnosis_with_hcc")

# Join and calculate member HCCs
member_hccs = claims \
    .join(members, "member_id") \
    .join(diagnosis_with_hcc, "claim_id") \
    .filter(col("is_hcc") == 1) \
    .withColumn("age", year(current_date()) - year(col("birth_date"))) \
    .select(
        "member_id", "first_name", "last_name", "birth_date", "gender", 
        "plan_id", "age", "hcc_category", "hcc_coefficient", "diagnosis_code"
    ) \
    .distinct()

# Calculate demographic scores and aggregate HCC scores
member_risk_scores_pyspark = member_hccs \
    .withColumn("demographic_score",
        when(col("age") < 65, 0.350)
        .when(col("age") <= 69, 0.450)
        .when(col("age") <= 74, 0.550)
        .when(col("age") <= 79, 0.650)
        .otherwise(0.750)
    ) \
    .groupBy("member_id", "first_name", "last_name", "birth_date", 
             "gender", "plan_id", "age", "demographic_score") \
    .agg(
        _sum("hcc_coefficient").alias("hcc_score"),
        countDistinct("hcc_category").alias("hcc_count"),
        collect_set("hcc_category").alias("hcc_categories"),
        collect_set("diagnosis_code").alias("diagnosis_codes")
    ) \
    .withColumn("total_risk_score", col("demographic_score") + col("hcc_score")) \
    .withColumn("projected_annual_payment", 
                spark_round(col("total_risk_score") * 10000, 2)) \
    .orderBy(col("total_risk_score").desc())

# Display results
print("📊 PySpark Risk Score Calculation - Top 10 Members:")
display(member_risk_scores_pyspark.limit(10))

# Optionally write to table
# member_risk_scores_pyspark.write.format("delta").mode("overwrite") \
#     .saveAsTable("payer_gold.member_risk_scores_pyspark")


### 📊 View Risk Score Results

Let's see the calculated risk scores:


In [0]:
%sql
-- View top 20 members by risk score
SELECT 
  member_id,
  CONCAT(first_name, ' ', last_name) as member_name,
  age,
  gender,
  hcc_count,
  ROUND(demographic_score, 3) as demo_score,
  ROUND(hcc_score, 3) as hcc_score,
  ROUND(total_risk_score, 3) as risk_score,
  projected_annual_payment
FROM payer_gold.member_risk_scores
ORDER BY total_risk_score DESC
LIMIT 20;


### 📊 SAS vs. Databricks SQL Comparison

#### Traditional SAS Approach

```sas
/* SAS: Revenue Forecast by Plan */
PROC SQL;
    CREATE TABLE work.revenue_forecast AS
    SELECT 
        plan_id,
        COUNT(DISTINCT member_id) AS total_members,
        ROUND(AVG(total_risk_score), 0.001) AS avg_risk_score,
        ROUND(MIN(total_risk_score), 0.001) AS min_risk_score,
        ROUND(MAX(total_risk_score), 0.001) AS max_risk_score,
        SUM(projected_annual_payment) AS total_projected_revenue,
        ROUND(AVG(projected_annual_payment), 0.01) AS avg_payment_per_member,
        SUM(CASE WHEN total_risk_score >= 1.5 THEN 1 ELSE 0 END) AS high_risk_members,
        SUM(CASE WHEN total_risk_score < 1.0 THEN 1 ELSE 0 END) AS low_risk_members
    FROM work.member_risk_scores
    GROUP BY plan_id
    ORDER BY total_projected_revenue DESC;
QUIT;

/* Export to Excel for reporting */
PROC EXPORT DATA=work.revenue_forecast
    OUTFILE='/path/to/revenue_forecast.xlsx'
    DBMS=XLSX REPLACE;
RUN;

/* Generate summary report */
PROC PRINT DATA=work.revenue_forecast;
    TITLE 'Revenue Forecast by Plan';
RUN;
```

**SAS Limitations:**
- ❌ Manual export steps for reporting
- ❌ No real-time dashboard integration
- ❌ Limited to single-server processing
- ❌ Requires additional tools for visualization
- ❌ Static reports need manual refresh

#### Databricks SQL Approach



## Example 2: Revenue Forecast & Impact Analysis

### 💰 Business Goal
Project total CMS revenue based on risk scores to support financial planning.

**Key Metrics:**
- Total member population
- Average risk score
- Projected annual revenue
- Revenue by plan and risk tier


In [0]:
%sql
-- Revenue Forecast by Plan
CREATE OR REPLACE TABLE payer_gold.revenue_forecast AS
SELECT
  plan_id,
  COUNT(DISTINCT member_id) as total_members,
  ROUND(AVG(total_risk_score), 3) as avg_risk_score,
  ROUND(MIN(total_risk_score), 3) as min_risk_score,
  ROUND(MAX(total_risk_score), 3) as max_risk_score,
  SUM(projected_annual_payment) as total_projected_revenue,
  ROUND(AVG(projected_annual_payment), 2) as avg_payment_per_member,
  SUM(CASE WHEN total_risk_score >= 1.5 THEN 1 ELSE 0 END) as high_risk_members,
  SUM(CASE WHEN total_risk_score < 1.0 THEN 1 ELSE 0 END) as low_risk_members
FROM payer_gold.member_risk_scores
GROUP BY plan_id
ORDER BY total_projected_revenue DESC;


In [0]:
%sql
-- Display revenue forecast
SELECT * FROM payer_gold.revenue_forecast;


### 📊 SAS vs. Databricks SQL Comparison

#### Traditional SAS Approach

```sas
/* SAS: HCC Distribution Analysis */
/* Step 1: Unnest HCC categories from member_risk_scores */
/* Note: SAS doesn't have native array explosion like SQL EXPLODE */
/* Must use DATA step with ARRAY processing */

DATA work.hcc_exploded;
    SET work.member_risk_scores;
    ARRAY hccs hcc_cat1-hcc_cat10;  /* Assumes max 10 HCCs per member */
    
    DO i = 1 TO DIM(hccs);
        IF hccs[i] NE . THEN DO;
            hcc_category = hccs[i];
            OUTPUT;
        END;
    END;
    DROP hcc_cat1-hcc_cat10 i;
RUN;

/* Step 2: Join with HCC reference and aggregate */
PROC SQL;
    CREATE TABLE work.hcc_distribution AS
    SELECT 
        he.hcc_category,
        r.diagnosis_desc,
        r.hcc_coefficient,
        COUNT(DISTINCT he.member_id) AS member_count,
        COUNT(DISTINCT he.plan_id) AS plan_count,
        ROUND(r.hcc_coefficient * COUNT(DISTINCT he.member_id) * 10000, 0.01) 
            AS total_revenue_impact,
        ROUND(r.hcc_coefficient * 10000, 0.01) AS revenue_per_member
    FROM work.hcc_exploded AS he
    INNER JOIN work.hcc_reference AS r 
        ON he.hcc_category = r.hcc_category
    GROUP BY he.hcc_category, r.diagnosis_desc, r.hcc_coefficient
    ORDER BY total_revenue_impact DESC;
QUIT;
```

**SAS Challenges:**
- ❌ No native EXPLODE function - requires manual array processing
- ❌ Must pre-define array size (max HCCs per member)
- ❌ Two-step process: DATA step + PROC SQL
- ❌ Complex logic for dynamic array sizes
- ❌ Performance issues with large datasets

#### Databricks SQL Approach - Single Statement!



## Example 3: HCC Distribution Analysis

### 📈 Business Goal
Understand which HCC categories drive the most revenue and identify coding opportunities.

This helps HQRI:
- Identify high-value diagnoses for provider education
- Monitor HCC capture rates
- Find gaps in documentation


In [0]:
%sql
-- HCC Category Distribution and Revenue Impact
CREATE OR REPLACE TABLE payer_gold.hcc_distribution AS
WITH hcc_exploded AS (
  SELECT
    member_id,
    plan_id,
    EXPLODE(hcc_categories) as hcc_category
  FROM payer_gold.member_risk_scores
)
SELECT
  he.hcc_category,
  r.diagnosis_desc,
  r.hcc_coefficient,
  COUNT(DISTINCT he.member_id) as member_count,
  COUNT(DISTINCT he.plan_id) as plan_count,
  ROUND(r.hcc_coefficient * COUNT(DISTINCT he.member_id) * 10000, 2) as total_revenue_impact,
  ROUND(r.hcc_coefficient * 10000, 2) as revenue_per_member
FROM hcc_exploded he
INNER JOIN payer_gold.hcc_reference r ON he.hcc_category = r.hcc_category
GROUP BY he.hcc_category, r.diagnosis_desc, r.hcc_coefficient
ORDER BY total_revenue_impact DESC;

In [0]:
%sql
-- View HCC distribution
SELECT * FROM payer_gold.hcc_distribution;



## Example 4: Data Quality & Compliance Audit

### ✅ Business Goal
Ensure encounter data meets CMS submission standards and identify data quality issues.

**CMS Requirements:**
- Valid diagnosis codes (ICD-10 format)
- Complete member demographics
- Valid provider NPIs
- Service dates within coverage period


### 📊 SAS vs. Databricks SQL Comparison

#### Traditional SAS Approach

```sas
/* SAS: Data Quality Audit - Requires Multiple Queries */

/* Query 1: Total Encounters */
PROC SQL;
    CREATE TABLE work.audit_total AS
    SELECT 
        'Total Encounters' AS metric,
        COUNT(*) AS record_count,
        . AS pct_of_total,
        'INFO' AS severity
    FROM work.claims;
QUIT;

/* Query 2: Encounters Missing Diagnosis */
PROC SQL;
    CREATE TABLE work.audit_missing_dx AS
    SELECT 
        'Encounters Missing Diagnosis' AS metric,
        COUNT(DISTINCT c.claim_id) AS record_count,
        CALCULATED record_count * 100.0 / 
            (SELECT COUNT(*) FROM work.claims) AS pct_of_total,
        'ERROR' AS severity
    FROM work.claims AS c
    LEFT JOIN work.diagnosis_raw AS d 
        ON c.claim_id = d.claim_id
    WHERE d.claim_id IS NULL;
QUIT;

/* Query 3: HCC Mapping Rate */
PROC SQL;
    CREATE TABLE work.audit_hcc_mapped AS
    SELECT 
        'Diagnoses Mapped to HCC' AS metric,
        COUNT(*) AS record_count,
        CALCULATED record_count * 100.0 / 
            (SELECT COUNT(*) FROM work.diagnosis_raw) AS pct_of_total,
        'INFO' AS severity
    FROM work.diagnosis_with_hcc
    WHERE is_hcc = 1;
QUIT;

/* Repeat for remaining checks... */

/* Combine all audit results */
DATA work.data_quality_audit;
    SET work.audit_total
        work.audit_missing_dx
        work.audit_hcc_mapped
        /* ... other audit tables ... */;
        
    /* Add status flag */
    IF severity = 'ERROR' AND record_count > 0 THEN status = 'FAIL';
    ELSE IF severity = 'WARNING' AND pct_of_total > 5 THEN status = 'REVIEW';
    ELSE status = 'PASS';
RUN;

/* Sort and display */
PROC SORT DATA=work.data_quality_audit;
    BY severity DESCENDING pct_of_total;
RUN;
```

**SAS Challenges:**
- ❌ Must create separate queries for each audit check
- ❌ Manual combination with DATA step
- ❌ Multiple intermediate tables clutter workspace
- ❌ Difficult to maintain as audit rules grow
- ❌ No UNION ALL equivalent in single PROC SQL
- ❌ Subquery limitations in SELECT clause

#### Databricks SQL Approach - Elegant & Maintainable!


In [0]:
%sql
-- Data Quality Audit for CMS Submission
CREATE OR REPLACE TABLE payer_gold.data_quality_audit AS
WITH encounter_checks AS (
  SELECT
    'Total Encounters' as metric,
    COUNT(*) as record_count,
    NULL as pct_of_total,
    'INFO' as severity
  FROM payer_silver.claims

  UNION ALL

  SELECT
    'Encounters Missing Diagnosis',
    COUNT(DISTINCT c.claim_id),
    ROUND(
      COUNT(DISTINCT c.claim_id) * 100.0 / (SELECT COUNT(*) FROM payer_silver.claims),
      2
    ),
    'ERROR'
  FROM payer_silver.claims c
  LEFT JOIN payer_bronze.diagnosis_raw d ON c.claim_id = d.claim_id
  WHERE d.claim_id IS NULL

  UNION ALL

  SELECT
    'Diagnoses Mapped to HCC',
    COUNT(*),
    ROUND(
      COUNT(*) * 100.0 / (SELECT COUNT(*) FROM payer_bronze.diagnosis_raw),
      2
    ),
    'INFO'
  FROM payer_gold.diagnosis_with_hcc
  WHERE is_hcc = 1

  UNION ALL

  SELECT
    'Members Without Risk Scores',
    COUNT(DISTINCT m.member_id),
    ROUND(
      COUNT(DISTINCT m.member_id) * 100.0 / (SELECT COUNT(*) FROM payer_silver.members),
      2
    ),
    'WARNING'
  FROM payer_silver.members m
  LEFT JOIN payer_gold.member_risk_scores r ON m.member_id = r.member_id
  WHERE r.member_id IS NULL

  UNION ALL

  SELECT
    'Providers Missing NPI',
    COUNT(*),
    ROUND(
      COUNT(*) * 100.0 / (SELECT COUNT(*) FROM payer_silver.providers),
      2
    ),
    'ERROR'
  FROM payer_silver.providers
  WHERE npi IS NULL OR TRIM(npi) = ''

  UNION ALL

  SELECT
    'Claims with Invalid Status',
    COUNT(*),
    ROUND(
      COUNT(*) * 100.0 / (SELECT COUNT(*) FROM payer_silver.claims),
      2
    ),
    'WARNING'
  FROM payer_silver.claims
  WHERE claim_status NOT IN ('approved', 'paid', 'pending')
)
SELECT
  metric,
  record_count,
  COALESCE(pct_of_total, 0.0) as pct_of_total,
  severity,
  CASE
    WHEN severity = 'ERROR' AND record_count > 0 THEN 'FAIL'
    WHEN severity = 'WARNING' AND pct_of_total > 5 THEN 'REVIEW'
    ELSE 'PASS'
  END as status
FROM encounter_checks
ORDER BY
  CASE severity WHEN 'ERROR' THEN 1 WHEN 'WARNING' THEN 2 ELSE 3 END,
  pct_of_total DESC;

In [0]:
%sql
-- View data quality audit results
SELECT * FROM payer_gold.data_quality_audit;


## Example 5: Member Risk Stratification

### 👥 Business Goal
Segment members by risk level to support care management and intervention programs.

**Risk Tiers:**
- **Very High Risk** (Score > 2.0): Intensive care management
- **High Risk** (Score 1.5-2.0): Enhanced monitoring
- **Moderate Risk** (Score 1.0-1.5): Standard care
- **Low Risk** (Score < 1.0): Preventive care focus


In [0]:
%sql
-- Member Risk Stratification
CREATE OR REPLACE TABLE payer_gold.member_risk_stratification AS
SELECT
  member_id,
  CONCAT(first_name, ' ', last_name) as member_name,
  age,
  gender,
  plan_id,
  total_risk_score,
  hcc_count,
  projected_annual_payment,
  CASE
    WHEN total_risk_score >= 2.0 THEN 'Very High Risk'
    WHEN total_risk_score >= 1.5 THEN 'High Risk'
    WHEN total_risk_score >= 1.0 THEN 'Moderate Risk'
    ELSE 'Low Risk'
  END as risk_tier,
  CASE
    WHEN total_risk_score >= 2.0 THEN 'Intensive Care Management Required'
    WHEN total_risk_score >= 1.5 THEN 'Enhanced Monitoring Recommended'
    WHEN total_risk_score >= 1.0 THEN 'Standard Care Protocol'
    ELSE 'Preventive Care Focus'
  END as care_recommendation,
  hcc_categories as active_hcc_list
FROM payer_gold.member_risk_scores;


In [0]:
%sql
-- Summary statistics by risk tier
SELECT
  risk_tier,
  COUNT(*) as member_count,
  ROUND(AVG(age), 1) as avg_age,
  ROUND(AVG(total_risk_score), 3) as avg_risk_score,
  ROUND(AVG(hcc_count), 1) as avg_hcc_count,
  SUM(projected_annual_payment) as total_revenue,
  ROUND(AVG(projected_annual_payment), 2) as avg_payment_per_member
FROM payer_gold.member_risk_stratification
GROUP BY risk_tier
ORDER BY avg_risk_score DESC;


### 📊 SAS vs. PySpark Comparison

#### Traditional SAS Approach

```sas
/* SAS: Provider Performance on Risk Capture */
PROC SQL;
    CREATE TABLE work.provider_performance AS
    SELECT 
        p.provider_id,
        p.provider_name,
        p.specialty,
        p.city,
        p.state,
        COUNT(DISTINCT c.member_id) AS unique_members,
        COUNT(c.claim_id) AS total_encounters,
        ROUND(AVG(m.total_risk_score), 0.001) AS avg_member_risk_score,
        SUM(dh.hcc_coefficient) AS total_hcc_value,
        COUNT(DISTINCT dh.hcc_category) AS unique_hccs_captured,
        ROUND(SUM(m.projected_annual_payment), 0.01) AS attributed_revenue,
        ROUND(COUNT(DISTINCT dh.hcc_category) / COUNT(DISTINCT c.member_id), 0.01) 
            AS hcc_capture_rate
    FROM work.claims AS c
    INNER JOIN work.providers AS p 
        ON c.provider_id = p.provider_id
    INNER JOIN work.member_risk_scores AS m 
        ON c.member_id = m.member_id
    INNER JOIN work.diagnosis_with_hcc AS dh 
        ON c.claim_id = dh.claim_id
    GROUP BY p.provider_id, p.provider_name, p.specialty, p.city, p.state
    ORDER BY attributed_revenue DESC;
QUIT;

/* Create top providers report */
PROC PRINT DATA=work.provider_performance(OBS=20);
    TITLE 'Top 20 Providers by Risk Capture Performance';
    VAR provider_name specialty unique_members avg_member_risk_score 
        unique_hccs_captured attributed_revenue hcc_capture_rate;
RUN;
```

**SAS Limitations:**
- ❌ Must calculate derived metrics (hcc_capture_rate) in the same SELECT
- ❌ No withColumn() equivalent for cleaner syntax
- ❌ Limited to single-server memory for large joins
- ❌ Static output - no interactive display
- ❌ Requires separate PROC PRINT for visualization

#### PySpark Approach - More Flexible & Scalable!



## Example 6: Provider Performance on Risk Capture

### 🏥 Business Goal
Identify which providers excel at documenting HCC conditions to guide provider education.

**Key Metrics:**
- Members per provider
- Average risk score of provider's panel
- HCC capture rate
- Revenue attributed to provider


In [0]:
from pyspark.sql.functions import count, countDistinct, avg, sum, round as spark_round, col

# Provider Performance Analysis
claims = spark.table("payer_silver.claims")
providers = spark.table("payer_silver.providers")
members = spark.table("payer_gold.member_risk_scores")
diagnosis_hcc = spark.table("payer_gold.diagnosis_with_hcc")

# Join claims with risk scores
provider_performance = claims \
    .join(providers, "provider_id") \
    .join(members, "member_id") \
    .join(diagnosis_hcc, "claim_id") \
    .groupBy(
        "provider_id",
        "provider_name",
        "specialty",
        "city",
        "state"
    ) \
    .agg(
        countDistinct("member_id").alias("unique_members"),
        count("claim_id").alias("total_encounters"),
        spark_round(avg("total_risk_score"), 3).alias("avg_member_risk_score"),
        sum(col("hcc_coefficient")).alias("total_hcc_value"),
        countDistinct(col("hcc_category")).alias("unique_hccs_captured"),
        spark_round(sum("projected_annual_payment"), 2).alias("attributed_revenue")
    ) \
    .withColumn(
        "hcc_capture_rate",
        spark_round(col("unique_hccs_captured") / col("unique_members"), 2)
    ) \
    .orderBy(col("attributed_revenue").desc())

# Display top providers
print("🏥 Top 20 Providers by Risk Capture Performance:")
display(provider_performance.limit(20))

# Save to Gold table
provider_performance.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("payer_gold.provider_risk_capture_performance")



## Example 7: Encounter Datamart for CMS Submission

### 📊 Business Goal
Create a CMS-ready encounter datamart with all required fields and validations.

**CMS Submission Requirements:**
- Valid member enrollment
- Complete encounter details (dates, provider, diagnosis)
- Proper diagnosis code formatting (ICD-10)
- Service within coverage period
- Provider has valid NPI



## 🔄 Migration Summary: SAS to Databricks

### Key Advantages of Databricks Over SAS for Gold Layer Analytics

| Feature | SAS | Databricks SQL/PySpark |
|---------|-----|------------------------|
| **Array Operations** | Manual ARRAY processing | Native COLLECT_SET(), EXPLODE() |
| **Query Complexity** | Multiple PROC SQL steps | Single CTE-based queries |
| **Scalability** | Single-server memory limits | Distributed processing (petabyte scale) |
| **Modern Functions** | Limited window functions | Full SQL:2016 compliance |
| **Real-time Analytics** | Batch-only | Streaming + batch unified |
| **Visualization** | Separate tools (PROC GPLOT, SAS VA) | Built-in interactive dashboards |
| **Governance** | Manual security setup | Unity Catalog (row/column security) |
| **Version Control** | Limited | Full Delta Lake time travel |
| **Cloud Native** | Legacy architecture | Modern cloud-optimized |
| **Cost** | Expensive licensing | Pay-per-use consumption |
| **Programming** | SAS language only | SQL, Python, R, Scala, Java |

### 💡 Migration Best Practices

1. **Start with Simple Queries**: Begin with straightforward PROC SQL → Databricks SQL translations
2. **Leverage CTEs**: Replace multi-step SAS DATA/PROC steps with Common Table Expressions
3. **Use PySpark for Complex Logic**: When business rules are complex, PySpark offers more flexibility than SAS macros
4. **Embrace Modern Functions**: COLLECT_SET, EXPLODE, and window functions simplify code
5. **Unity Catalog = SAS Library**: Map SAS libraries to Unity Catalog schemas
6. **Delta Lake = SAS Datasets**: Delta tables provide ACID transactions like SAS datasets, but at scale

---


In [0]:
%sql
-- Sunmin: what is this?

-- Encounter Datamart for CMS Submission
CREATE OR REPLACE TABLE payer_gold.encounter_datamart_cms AS
SELECT
  -- Encounter identifiers
  c.claim_id as encounter_id,
  c.claim_date as encounter_date,
  c.claim_status as encounter_status,
  
  -- Member information
  m.member_id,
  m.first_name,
  m.last_name,
  m.birth_date,
  m.gender,
  m.plan_id,
  YEAR(CURRENT_DATE()) - YEAR(m.birth_date) as member_age,
  
  -- Provider information
  p.provider_id,
  p.npi as provider_npi,
  p.provider_name,
  p.specialty as provider_specialty,
  p.state as provider_state,
  
  -- Diagnosis information
  d.diagnosis_code as icd10_code,
  d.diagnosis_desc,
  d.hcc_category,
  d.hcc_coefficient,
  d.is_hcc,
  
  -- Claim financial
  c.total_charge,
  
  -- Data quality flags
  CASE 
    WHEN m.member_id IS NULL THEN 'FAIL: Missing Member'
    WHEN p.npi IS NULL OR TRIM(p.npi) = '' THEN 'FAIL: Missing Provider NPI'
    WHEN d.diagnosis_code IS NULL THEN 'FAIL: Missing Diagnosis'
    WHEN c.claim_date < m.effective_date THEN 'FAIL: Service Before Coverage'
    WHEN c.claim_status NOT IN ('approved', 'paid') THEN 'WARNING: Invalid Status'
    ELSE 'PASS'
  END as submission_validation_status,
  
  -- Submission flag
  CASE 
    WHEN m.member_id IS NOT NULL 
     AND p.npi IS NOT NULL 
     AND TRIM(p.npi) != ''
     AND d.diagnosis_code IS NOT NULL
     AND c.claim_date >= m.effective_date
     AND c.claim_status IN ('approved', 'paid')
    THEN 1 
    ELSE 0 
  END as cms_submission_ready,
  
  CURRENT_TIMESTAMP() as datamart_created_at
  
FROM payer_silver.claims c
INNER JOIN payer_silver.members m ON c.member_id = m.member_id
INNER JOIN payer_silver.providers p ON c.provider_id = p.provider_id
LEFT JOIN payer_gold.diagnosis_with_hcc d ON c.claim_id = d.claim_id;


In [0]:
%sql
-- Sunmin: what is this?

-- CMS Submission Readiness Summary
SELECT
  submission_validation_status,
  COUNT(*) as encounter_count,
  ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total,
  SUM(cms_submission_ready) as ready_for_submission
FROM payer_gold.encounter_datamart_cms
GROUP BY submission_validation_status
ORDER BY encounter_count DESC;


# AI/BI

Intelligent analytics for everyone!

Databricks AI/BI is a new type of business intelligence product designed to provide a deep understanding of your data's semantics, enabling self-service data analysis for everyone in your organization. AI/BI is built on a compound AI system that draws insights from the full lifecycle of your data across the Databricks platform, including ETL pipelines, lineage, and other queries.

<img src="https://www.databricks.com/sites/default/files/2025-05/hero-image-ai-bi-v2-2x.png?v=1748417271" alt="Managed Tables" width="600" height="500">

# Genie

Talk with your data

Now everyone can get insights from data simply by asking questions in natural language.

<img src="https://www.databricks.com/sites/default/files/2025-06/ai-bi-genie-hero.png?v=1749162682" alt="Managed Tables" width="600" height="500">


# 🎓 Workshop Summary & Next Steps

## 🎉 Congratulations!

You've completed the Databricks Healthcare Payer Analytics Workshop! Let's review what you learned:

---

## ✅ What You Accomplished

### 1. **Medallion Architecture**
- ✅ Built a complete **Bronze → Silver → Gold** pipeline
- ✅ Understood data quality improvement at each layer
- ✅ Created analytics-ready datasets

### 2. **Data Engineering Skills**
- ✅ Loaded data using **COPY INTO**
- ✅ Transformed data with **SQL and PySpark**
- ✅ Applied data quality checks and validations
- ✅ Created aggregations and derived metrics

### 3. **Analytics & Visualization**
- ✅ Generated business insights from data
- ✅ Created interactive visualizations
- ✅ Performed statistical analysis
- ✅ Built executive dashboards

### 4. **Databricks Platform**
- ✅ Worked with **Unity Catalog**
- ✅ Used **Delta Lake** for reliable data storage
- ✅ Leveraged **AI Assistant** for code help
- ✅ Applied performance optimization techniques

### 5. **Refer to the Best Practices!**
Best practices notebook: _**[Reference] Best Practices**_

---

## 📝 Feedback

We'd love to hear your thoughts on this workshop!

**What worked well?** What could be improved? **What topics do you want to learn next?**

---

## 🙏 Thank You!

Thank you for participating in this workshop. We hope you found it valuable and are excited to continue your Databricks journey! 🚀

---
